# Stage 04 — Data Acquisition & Ingestion

**Goals**
- API pull (with `.env` for key if needed) → validate → save to `data/raw/`
- Scrape a small public table with BeautifulSoup → validate → save to `data/raw/`
- Docs: sources/params/validation + assumptions & risks

In [1]:
import os; print(os.getcwd())
from pathlib import Path
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup



/Users/chen/bootcamp_tingchen_chen/homework/homework4/notebooks


/opt/miniconda3/envs/fe-course/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
# Cell 2：設定匯入與路徑（從 notebooks/ 相對到 ../src 與 ../data/raw）
from pathlib import Path
import sys, os, importlib

NB_DIR = Path.cwd()                 # .../homework/homework4/notebooks
HW4_DIR = NB_DIR.parent             # .../homework/homework4
SRC_PATH = (HW4_DIR / "src").resolve()
RAW_DIR = (HW4_DIR / "data" / "raw").resolve()

# 讓專案的 src 走在最前面，避免撞到 site-packages 的 utils
sys.path.insert(0, str(SRC_PATH))
importlib.invalidate_caches()

print("cwd:", NB_DIR)
print("SRC_PATH:", SRC_PATH, "exists?", SRC_PATH.exists())
print("RAW_DIR:", RAW_DIR)

# 匯入你自己的工具函式（檔名是 hw4_utils.py）
from hw4_utils import load_api_key, ensure_dtypes, validate_df, save_raw_csv

# 建立 raw 資料夾
RAW_DIR.mkdir(parents=True, exist_ok=True)



cwd: /Users/chen/bootcamp_tingchen_chen/homework/homework4/notebooks
SRC_PATH: /Users/chen/bootcamp_tingchen_chen/homework/homework4/src exists? True
RAW_DIR: /Users/chen/bootcamp_tingchen_chen/homework/homework4/data/raw


In [4]:
import pandas as pd
import numpy as np
import yfinance as yf

TICKER   = "AAPL"    # 可改
PERIOD   = "1y"
INTERVAL = "1d"

# 1) 抓資料（固定不自動調整，保留 Adj Close 欄；也關掉進度條）
data = yf.download(TICKER, period=PERIOD, interval=INTERVAL, auto_adjust=False, progress=False)

# 2) 若是 MultiIndex 欄位就攤平；否則直接用
if isinstance(data.columns, pd.MultiIndex):
    data.columns = ['_'.join([str(x) for x in tup if x]).strip().lower() for tup in data.columns]
else:
    data.columns = [str(c).strip().lower().replace(' ', '_') for c in data.columns]

# 3) reset index 並把日期欄標準化
df_api = data.reset_index()
df_api.rename(columns={"date": "date", "datetime": "date"}, inplace=True)
# yfinance 可能把日期欄叫 "date" 或 "index"；補救：
if "date" not in df_api.columns and "index" in df_api.columns:
    df_api.rename(columns={"index": "date"}, inplace=True)

# 4) 把常見價量欄位對齊成一套命名（若原本就小寫，這步不會動到）
rename_map = {
    "open": "open",
    "high": "high",
    "low": "low",
    "close": "close",
    "adj_close": "adj_close",
    "volume": "volume",
    "open_aapl": "open", "high_aapl": "high", "low_aapl": "low",
    "close_aapl": "close", "adj_close_aapl": "adj_close", "volume_aapl": "volume",
}
df_api.rename(columns={k: v for k, v in rename_map.items() if k in df_api.columns}, inplace=True)

# 5) 只保留我們關心的欄（存在才留）
wanted = ["date", "open", "high", "low", "close", "adj_close", "volume"]
present = [c for c in wanted if c in df_api.columns]
df_api = df_api.loc[:, present]

# 6) 型別處理（只對真的存在的欄位做）
float_cols = [c for c in ["open", "high", "low", "close", "adj_close"] if c in df_api.columns]
df_api = ensure_dtypes(df_api, date_cols=["date"], float_cols=float_cols)

# 7) 驗證（至少 5 列、必要欄位以實際存在為準）
required = [c for c in ["date", "open", "high", "low", "close", "volume"] if c in df_api.columns]
report_api = validate_df(df_api, required_cols=required, min_rows=5)
print("columns:", df_api.columns.tolist())
report_api



columns: ['open', 'high', 'low', 'close', 'volume']


{'ok': True,
 'missing_cols': [],
 'na_counts': {'open': 0, 'high': 0, 'low': 0, 'close': 0, 'volume': 0},
 'shape': (251, 5)}

In [5]:
api_path = save_raw_csv(df_api, kind="api", source="yfinance", tag=TICKER, outdir=RAW_DIR)
api_path


PosixPath('/Users/chen/bootcamp_tingchen_chen/homework/homework4/data/raw/api_yfinance_aapl_20250819-1024.csv')

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"  # 可換其他允許公開的表格頁
SITE = "wikipedia"
TAG  = "sp500_table"

html = requests.get(URL, timeout=30).text
soup = BeautifulSoup(html, "lxml")
table = soup.find("table", {"class": "wikitable"})  # 依頁面調整 selector
assert table is not None, "找不到表格，請檢查 selector"

# headers
ths = table.find("tr").find_all(["th", "td"])
columns = [th.get_text(strip=True) for th in ths]

# rows
rows = []
for tr in table.find_all("tr")[1:]:
    tds = tr.find_all(["td","th"])
    if not tds:
        continue
    rows.append([td.get_text(strip=True) for td in tds])

df_scrape = pd.DataFrame(rows, columns=columns[:len(rows[0])] if rows else columns)

# 簡單型別處理（日期欄、數值欄可依實際欄名調整）
for c in df_scrape.columns:
    if "date" in c.lower():
        df_scrape[c] = pd.to_datetime(df_scrape[c], errors="coerce")
df_scrape.head(3)


,Symbol,Security,GICSSector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,0000066740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,0000001800,1888


In [7]:
required_cols = df_scrape.columns[:3].tolist()   # 至少要求幾個欄位存在；可改成你需要的欄名
report_scrape = validate_df(df_scrape, required_cols=required_cols, min_rows=5)
report_scrape

scrape_path = save_raw_csv(df_scrape, kind="scrape", source=SITE, tag=TAG, outdir=RAW_DIR)
scrape_path

PosixPath('/Users/chen/bootcamp_tingchen_chen/homework/homework4/data/raw/scrape_wikipedia_sp500_table_20250819-1024.csv')

## Sources, Parameters, and Validation

**API**
- Source: yfinance  
- Parameters: ticker=`AAPL`, period=`1y`, interval=`1d`, auto_adjust=False  
- Validation: required columns = [`date`, `open`, `high`, `low`, `close`, `volume`]; checked with `report_api` (row count, NA values).  

**Scrape**
- URL: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies  
- Selector: `<table class="wikitable">` (first occurrence)  
- Validation: required columns = first three columns of the table; checked with `report_scrape` (row count, NA values).  

**Reproducible Filenames**
- `data/raw/api_yfinance_AAPL_YYYYMMDD-HHMM.csv`  
- `data/raw/scrape_wikipedia_sp500_table_YYYYMMDD-HHMM.csv`



## Assumptions & Risks
- API schema and column names may change over time; cleaning and validation logic must be updated accordingly.  
- Scraping depends on the HTML structure (table class and column order); if the webpage changes, selectors and parsing logic must be revised.  
- `.env` file is for local use only; only `.env.example` is included in the repo. Sensitive API keys must not be committed.  